<a href="https://colab.research.google.com/github/EvaaVos/Quantum_Computing_Course/blob/main/PennyLane/Challenge.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [59]:
!pip install pennylane

import json
import pennylane as qml
import pennylane.numpy as np

In [75]:
def W(params):
  if params.ndim == 3:
   for X in range(params.shape[0]):
    for i in range(params.shape[1]):
     for j in range(params.shape[2]):
        qml.RY(params[X,i,j], wires=j)
     for j in range(params.shape[2]):
      if j < params.shape[2] - 1 :
        qml.CNOT(wires=[j, j + 1])
      if j ==  params.shape[2] - 1:
        qml.CNOT(wires=[j, 0])
  if params.ndim == 2:
    for i in range(params.shape[0]):
     for j in range(params.shape[1]):
        qml.RY(params[i,j], wires=j)
     for j in range(params.shape[1]):
        if j < params.shape[1] - 1 :
         qml.CNOT(wires=[j, j + 1])
        if j == params.shape[1] - 1:
          qml.CNOT(wires=[j, 0])

def S(g,x,num_wires):
  for i in range(num_wires):
    H = qml.Hamiltonian([1.0],[g(wires=i)])
    qml.ApproxTimeEvolution(H, -x, 1)

dev = qml.device("default.qubit", wires = 10)

@qml.qnode(dev)
def quantum_model(param_set, g, x):
  if param_set.ndim == 3:
   num_mat,N,M = param_set.shape
  else:
   N,M = param_set.shape
  for X in range(num_mat):
   if X != num_mat-1:
    W(param_set[X])
    S(g, x, M)
   else:
    W(param_set[X])
    break
  return qml.probs(wires = [0])

# These functions are used to test your solution

def run(test_case_input: str) -> str:
    ins = json.loads(test_case_input)
    params = np.array(ins[0])
    g = getattr(qml, ins[1])
    x = ins[2]
    outs = quantum_model(params, g, x).tolist()
    #print(qml.draw(quantum_model)(params, g, x))
    return str(outs)

def check(solution_output: str, expected_output: str) -> None:
    solution_output = json.loads(solution_output)
    expected_output = json.loads(expected_output)
    print('Expected probabilities:' , expected_output)
    dev_test = qml.device("default.qubit", wires = [0,1,2])

    @qml.qnode(dev_test)
    def w_node(params):

        W(params)

        return qml.probs(wires = [0,1])

    @qml.qnode(dev_test)
    def s_node(g, x, num_wires):

        S(g, x, num_wires)

        return qml.probs(wires = [0,1])

    params_test = np.array([[np.pi, np.pi/4, np.pi],[np.pi, np.pi/4, np.pi/3]])
    w_test = w_node(params_test)

    s_test = s_node(qml.PauliX, np.pi/7, 3)

    assert np.allclose(w_test, [0.10983496, 0.21338835, 0.03661165, 0.64016504], atol = 1e-3), "Something isn't quite right with the trainable block."

    assert np.allclose(s_test, [0.65892978, 0.15281512, 0.15281512, 0.03543998], atol = 1e-3), "Something isn't quite right with the encoding block."

    assert np.allclose(solution_output,expected_output, atol = 1e-3), "Not the correct probabilities for the quantum model."



test_cases = [
    ('[[[[1.0472, 0.7854, 3.1416, 0.3927],[1.0472, 0.7854, 3.1416, 0.5236]],[[1.0472, 0.7854, 1.5708, 0.3927],[0.7854, 0.7854, 1.5708, 0.7854]]],"PauliX", 0.7854]', '[0.46653, 0.53347]'),
    ('[[[[0.62832, 0.3927, 1.0472, 0.7854],[0.7854, 0.31416, 0.62832, 0.5236]],[[0.31416, 0.7854, 0.7854, 0.3927],[0.31416, 0.3927, 0.31416, 0.3927]]],"PauliY", 0.5236]', '[0.68594, 0.31406]')
]

# This will run the public test cases locally
for i, (input_, expected_output) in enumerate(test_cases):
    print(f"Running test case {i} with input '{input_}'...")

    try:
        output = run(input_)
        print ('Probabilities wire 0:', output)

    except Exception as exc:
        print(f"Runtime Error. {exc}")

    else:
        if message := check(output, expected_output):
            print(f"Wrong Answer. Have: '{output}'. Want: '{expected_output}'.")

        else:
            print("Correct!")

Running test case 0 with input '[[[[1.0472, 0.7854, 3.1416, 0.3927],[1.0472, 0.7854, 3.1416, 0.5236]],[[1.0472, 0.7854, 1.5708, 0.3927],[0.7854, 0.7854, 1.5708, 0.7854]]],"PauliX", 0.7854]'...
Probabilities wire 0: [0.4665295442152107, 0.5334704557847874]
Expected probabilities: [0.46653, 0.53347]
Correct!
Running test case 1 with input '[[[[0.62832, 0.3927, 1.0472, 0.7854],[0.7854, 0.31416, 0.62832, 0.5236]],[[0.31416, 0.7854, 0.7854, 0.3927],[0.31416, 0.3927, 0.31416, 0.3927]]],"PauliY", 0.5236]'...
Probabilities wire 0: [0.6859411491032397, 0.314058850896761]
Expected probabilities: [0.68594, 0.31406]
Correct!
